<a href="https://colab.research.google.com/github/ulises1229/ToxicovigilanceTool/blob/main/code/ToxicovigilanceTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Toxicovigilance tool
+ Author: Ulises Olivares
+ Email: uolivares@unam.mx
+ Dec 21, 2021

## 1. Import dataframe

In [ ]:
# Import libraries
import pandas as pd

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split # Split data 

## 2. Import datasets

In [ ]:
prado = pd.read_csv("https://raw.githubusercontent.com/HpcDataLab/ToxicovigilanceTool/main/data/Daily_flight_activity_Prado.csv")
prado.head(100)


### 2.1 Descibe dataset

In [ ]:
prado.describe()

In [ ]:
print("Unique bees: ", len(prado.ID.unique()))

# Subdivide data 70% training 30% test
X_train, X_test, y_train, y_test = train_test_split(prado, prado.Treatment, test_size=0.3)

print("Number of registers:", prado.shape)

# Training data
print("Shape of data for training:",X_train.shape)

# Testing data
print("Shape of data for testing:",X_test.shape)

#print("Training data", X_train)

print("Unique bees: ", len(X_train.ID.unique()))

### 2.2 Subdivide registers into two subsets: Control and pesticides

In [ ]:
# Separete a single bee 
single = prado[prado.ID ==  "K 4152"] 

# Separate control bees
control = prado[prado.Treatment == "control"]

# Separate pesticide bees
pesticide = prado[prado.Treatment == "pesticide"]

print("Number of control registers: ", len(control))
print("Unique control bees",len(control.ID.unique()))

print("Number of pesticide registers: ", len(pesticide))
print("Unique pesticide bees",len(pesticide.ID.unique()))

print("Total registers for K 4150:",len(single))

# Subdivide data 70% training 30% test Control
C_X_train, C_X_test, C_y_train, C_y_test = train_test_split(control, control.Treatment, test_size=0.3)

P_X_train, P_X_test, P_y_train, P_y_test = train_test_split(pesticide, pesticide.Treatment, test_size=0.3)

print("Total training control registers: ",len(C_X_train))
print("Total training pesticide registers: ", len(P_X_train))

# merge testing and training datasets
X_train = pd.merge(C_X_train, P_X_train, how="outer")
X_test =  pd.merge(C_X_test, P_X_test, how="outer")

y_train = pd.merge(C_y_train, P_y_train, how="outer")
y_test =  pd.merge(C_y_test, P_y_test, how="outer")



print("Total train Regs.", len(X_train))
print("Total test Regs.", len(X_test))

#print(X_train)


In [ ]:
def build_model(trainSize):
    model = keras.Sequential()
    # Add an Embedding layer expecting input vocab of size 1000, and
    # output embedding dimension of size 64.
    model.add(layers.Embedding(input_dim=trainSize, output_dim=1024))

    # Add a LSTM layer with 128 internal units.
    model.add(layers.LSTM(1024))

    # Add a Dense layer with 2 units.
    model.add(layers.Dense(2))

    return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(X_train))

model.summary()